#**Step 2: Import All the Required Libraries**

In [2]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
# from peft import LoraConfig, PeftModel
# from trl import SFTTrainer

In [8]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"
# Fine-tuned model name
new_model = "Llama-2-7b-chat-finetune"

use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False


In [10]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
)
# model.config.use_cache = False
# model.config.pretraining_tp = 1

# # Load LLaMA tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training



Your GPU supports bfloat16: accelerate training with bf16=True


`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

C:\Users\rajab\miniconda3\envs\py3_120\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rajab\.cache\huggingface\hub\models--NousResearch--Llama-2-7b-chat-hf. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [11]:
model.save_pretrained(r"C:\Users\rajab\Desktop\stuffs\TimeLine\2024\09_Sep\models")


In [14]:
def quantization(model_name, output_dir, quant_params= None):
    if quant_params is None:
        
        use_4bit = True
        bnb_4bit_compute_dtype = "float16"
        bnb_4bit_quant_type = "nf4"
        use_nested_quant = False
        compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
        
        quant_params = BitsAndBytesConfig(
            load_in_4bit=use_4bit,
            bnb_4bit_quant_type=bnb_4bit_quant_type,
            bnb_4bit_compute_dtype=compute_dtype,
            bnb_4bit_use_double_quant=use_nested_quant,
        )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=quant_params,
    )
    model.save_pretrained(output_dir)
    return model
quantization(r"C:\Users\rajab\.cache\huggingface\hub\models--black-forest-labs--FLUX.1-schnell\snapshots\02046d1ec07fefab4733f28253ebee3a79dd4ea1\transformer", r"C:\Users\rajab\Desktop\stuffs\TimeLine\2024\09_Sep\models\flux")

ValueError: Unrecognized model in C:\Users\rajab\.cache\huggingface\hub\models--black-forest-labs--FLUX.1-schnell\snapshots\02046d1ec07fefab4733f28253ebee3a79dd4ea1\transformer. Should have a `model_type` key in its config.json, or contain one of the following strings in its name: albert, align, altclip, audio-spectrogram-transformer, autoformer, bark, bart, beit, bert, bert-generation, big_bird, bigbird_pegasus, biogpt, bit, blenderbot, blenderbot-small, blip, blip-2, bloom, bridgetower, bros, camembert, canine, chameleon, chinese_clip, chinese_clip_vision_model, clap, clip, clip_vision_model, clipseg, clvp, code_llama, codegen, cohere, conditional_detr, convbert, convnext, convnextv2, cpmant, ctrl, cvt, data2vec-audio, data2vec-text, data2vec-vision, dbrx, deberta, deberta-v2, decision_transformer, deformable_detr, deit, depth_anything, deta, detr, dinat, dinov2, distilbert, donut-swin, dpr, dpt, efficientformer, efficientnet, electra, encodec, encoder-decoder, ernie, ernie_m, esm, f

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from optimum.bnb import BnbQuantizer

def quantize_model(model_name, output_dir):
    # Load the model and tokenizer
    model = AutoModelForCausalLM.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Initialize the quantizer
    quantizer = BnbQuantizer(torch_dtype=torch.float16)

    # Quantize the model
    quantized_model = quantizer.quantize_model(model, tokenizer)

    # Save the quantized model
    quantized_model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

    print(f"Quantized model saved to: {output_dir}")

if __name__ == "__main__":
    model_name = "black-forest-labs/FLUX.1-schnell"
    output_dir = "./.quantized_flux_schnell"
    
    quantize_model(model_name, output_dir)

ModuleNotFoundError: No module named 'optimum.bnb'

In [11]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from huggingface_hub import hf_hub_download
import json
import os

def download_model_files(model_name):
    # Download the necessary files
    model_path = r'C:\Users\rajab\.cache\huggingface\hub\models--black-forest-labs--FLUX.1-schnell\snapshots\741f7c3ce8b383c54771c7003378a50191e9efe9\flux1-schnell.safetensors'
    config_path = hf_hub_download(repo_id=model_name, filename="model_index.json")
    
    return model_path, config_path

def quantize_model(model_name, output_dir):
    # Download model files
    model_path, config_path = download_model_files(model_name)

    # Load the config
    with open(config_path, 'r') as f:
        config = json.load(f)

    # Configure quantization
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    # Load the model with quantization config
    print(config_path)
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        config=config,
        quantization_config=bnb_config,
        trust_remote_code=True
    )
    # tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

    # Save the quantized model
    os.makedirs(output_dir, exist_ok=True)
    model.save_pretrained(output_dir)
    # tokenizer.save_pretrained(output_dir)

    print(f"Quantized model saved to: {output_dir}")

if __name__ == "__main__":
    model_name = "black-forest-labs/FLUX.1-schnell"
    output_dir = "./.quantized_flux_schnell"
    
    quantize_model(model_name, output_dir)

C:\Users\rajab\.cache\huggingface\hub\models--black-forest-labs--FLUX.1-schnell\snapshots\741f7c3ce8b383c54771c7003378a50191e9efe9\model_index.json


OSError: Incorrect path_or_model_id: 'C:\Users\rajab\.cache\huggingface\hub\models--black-forest-labs--FLUX.1-schnell\snapshots\741f7c3ce8b383c54771c7003378a50191e9efe9\flux1-schnell.safetensors'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

def load_llama_Model(base_model_path):
    tokenizer = AutoTokenizer.from_pretrained(base_model_path)
    model = AutoModelForCausalLM.from_pretrained(base_model_path)
    return model, tokenizer

base_model_path = r"C:\Users\rajab\Desktop\stuffs\TimeLine\2024\09_Sep\models\llama8b"
model7b, tokenizer7b = load_llama_Model(base_model_path)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


In [2]:

question = "What is the capital of France?"

# Step 1: Tokenize the input
input_ids = tokenizer7b.encode(question, return_tensors='pt')

# Step 2: Generate a response
with torch.no_grad():
    output = model7b.generate(input_ids, max_length=50, num_return_sequences=1)

# Step 3: Decode the response
response = tokenizer7b.decode(output[0], skip_special_tokens=True)

# Print the response
print(response)

C:\Users\rajab\miniconda3\envs\py3_120\Lib\site-packages\transformers\generation\utils.py:1850: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
C:\Users\rajab\miniconda3\envs\py3_120\Lib\site-packages\transformers\models\llama\modeling_llama.py:603: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


What is the capital of France?
 Hinweis: The answer is Paris.

What is the currency of France?
Hint: The currency of France is the Euro.

What is the largest city in France?
Hint: The


In [4]:

question = "What is the capital of Australia?"

# Step 1: Tokenize the input
input_ids = tokenizer7b.encode(question, return_tensors='pt')

# Step 2: Generate a response
with torch.no_grad():
    output = model7b.generate(input_ids, max_length=50, num_return_sequences=1)

# Step 3: Decode the response
response = tokenizer7b.decode(output[0], skip_special_tokens=True)

# Print the response
print(response)

What is the capital of Australia?
 Unterscheidung between "Capital" and "Town" or "City"

The capital of Australia is Canberra. Canberra is a city located in the Australian Capital Territory (ACT


In [1]:
35**2

1225

In [2]:
1225/60

20.416666666666668